Для начала просто глянем, как выглядят наши тренировочные данные.

In [17]:
import pandas as pd

train = pd.read_csv('train.csv')
#train.tail()
#train[(train['store_id'] == 164) & (train['store_id'] == 164)]
train['date'].unique()

array(['2021-07-29', '2021-07-30', '2021-07-31', '2021-08-01',
       '2021-08-02', '2021-08-03', '2021-08-04', '2021-08-05',
       '2021-08-06', '2021-08-07', '2021-08-08', '2021-08-09',
       '2021-08-10', '2021-08-11', '2021-08-12', '2021-08-13',
       '2021-08-14', '2021-08-15', '2021-08-16', '2021-08-17',
       '2021-08-18', '2021-08-19', '2021-08-20', '2021-08-21',
       '2021-08-22', '2021-08-23', '2021-08-24', '2021-08-25',
       '2021-08-26', '2021-08-27', '2021-08-28', '2021-08-29',
       '2021-08-30', '2021-08-31', '2021-09-01', '2021-09-02',
       '2021-09-03', '2021-09-04', '2021-09-05', '2021-09-06',
       '2021-09-07', '2021-09-08', '2021-09-09', '2021-09-10',
       '2021-09-11', '2021-09-12', '2021-09-13', '2021-09-14',
       '2021-09-15', '2021-09-16', '2021-09-17', '2021-09-18',
       '2021-09-19', '2021-09-20', '2021-09-21', '2021-09-22',
       '2021-09-23', '2021-09-24', '2021-09-25', '2021-09-26',
       '2021-09-27', '2021-09-28', '2021-09-29', '2021-

А ответ должен выглядеть примерно так

In [18]:
sample=pd.read_csv('sample_submission.csv')
sample.head()

,id,prediction
0,666677,22.0
1,666678,27.0
2,666679,26.0
3,666680,24.0
4,666681,42.0


"*fit будем делать по переменным id и sales. Голый id представляет собой связку (product_id, store_id, date)*" - я был молод и глуп, id сам по себе обьект никак не описывает. Обьект по сути есть комбинация фич, ничего более.

Не до конца известные данные выглядят вот так:

In [19]:
test=pd.read_csv('test.csv')
test.head()

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,pressure
0,666677,2022-02-14,Москва,1,1,1,4.79,облачно,87.3125,-1.9375,749.3125
1,666678,2022-02-15,Москва,1,1,1,4.79,переменная облачность,88.7500,-1.2500,752.6875
2,666679,2022-02-16,Москва,1,1,1,4.79,переменная облачность,90.3750,-1.5625,746.3125
3,666680,2022-02-17,Москва,1,1,1,4.79,"облачно, небольшой дождь",98.0000,1.7500,732.6875
4,666681,2022-02-18,Москва,1,1,1,4.79,"облачно, небольшие осадки",95.5000,1.3750,733.0000


Обработка данных.

1) Склеиваю оба датафрейма.

2) перевожу дни в правильный формат.

3) сортирую, как надо (и разбираюсь, почему так надо).

4) добавляю фичи по дню недели.

5) добавляю разумное количество лагов.

5.5) Обрубаю первые дни (к которым нельзя приписать лаги)

6) Обрабатываю категориальные признаки

7) Создаю датафрейм для обучения, т.е. выкидываю из текущего ненужные колонки по типу даты, id, инфы по погоде. очу обработать данные, пока не добавляя новые фичи.

In [20]:
from datetime import datetime

dtime1 = datetime(2021, 8, 17, 20)
dtime2 = datetime(2022, 1, 7, 0)
dtime2

datetime.datetime(2022, 1, 7, 0, 0)

In [21]:
from pandas.core.frame import DataFrame
from datetime import datetime

# 1
train['mode'] = 'train'
test['mode'] = 'test'
df = pd.concat([train, test])


# 3
df = df.sort_values(['store_id', 'product_id', 'date']).reset_index(drop=True)

# 2
df.date = pd.to_datetime(df.date)

# 4
df['weekday'] = df.date.dt.weekday
df['weekend'] = df['weekday'].isin([5, 6]).astype(int)


# 5
N=31 #количество лагов +-. канон 21
group = df.groupby(['store_id', 'product_id'])
for i in range(7, N):
    df[f'lag_day_{i}'] = group['sales'].shift(i)

# 5.5 
df_new=df

# не сработало, т.к. вррде есть вмагаз, которые только недавно открылись. они полны нанов, но первый мой метод их не удаляет
#dtime = datetime(2021, 8, 17, 20)
#df_new.drop(df_new[df_new.date<dtime].index, inplace=True) # Шикарно!!!

#df_new.groupby(['store_id', 'product_id']).apply(lambda group: group.iloc[21:])
df_new['num_in_group'] = df_new.groupby(['store_id', 'product_id']).cumcount()
df_new = df_new[df_new['num_in_group']>N-2]



# 6

df_new=pd.get_dummies(data=df_new, columns=['city_name'])
df_new=pd.get_dummies(data=df_new, columns=['product_id']) #Пробую после МАЕ 4.15
df_new=pd.get_dummies(data=df_new, columns=['weather_desc']) #Пробую после МАЕ 4.14
#погода и температура вообще не помогли

# 7

df_new.drop(['humidity','pressure','date'], axis=1, inplace=True)
df_new.drop(['price'], axis=1, inplace=True) # обсуждалось, что это уже содержится внутри product_id
df_new.drop(['num_in_group'], axis=1, inplace=True)





In [22]:
#pd.set_option('display.max_rows', None)
#df_new[(df_new['product_id'] == 1) & (df_new['store_id'] == 164)]

Попытка что-то обучить.

In [23]:
from numpy.ma.core import argsort
from sklearn.linear_model import LinearRegression
import numpy as np

temp=df_new.loc[df_new['mode'] == 'train']
arg=temp.drop(['mode','id','sales'], axis=1, inplace=False)
target=temp.drop(['mode','id'], axis=1, inplace=False)

#проверка, остались ли где-то наны
#arg[list(arg.columns)].isnull().sum()


LR = LinearRegression().fit(arg,target['sales'])

ValueError: ignored

Попытка что-то предсказать.

In [ ]:
temppp=df_new.loc[df_new['mode'] == 'test']
for_prediction=temppp.drop(['mode','id','sales'], axis=1, inplace=False)
res=temppp[['id','sales']]

res['sales'] = LR.predict(for_prediction)
res.rename(columns={"id":"id","sales": "prediction"}, inplace=True)

res.to_csv('result.csv', index=False, encoding='utf-8')